In [3]:
# Importação das bibliotecas necessárias
import pandas as pd

import requests

from pendulum import now

In [4]:
fonte = 'lapig'
dataset_name = 'pastagens_degradacao'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

ano_anterior = now().year - 1

### Bronze

In [ ]:
# Tabelas relativas ao vigor (degradação) das áreas de pastagem no Brasil

contador = 1

for ano in range(2000, ano_anterior):
    url = f'https://s3.lapig.iesa.ufg.br/ows/country/BRASIL/csv/pasture_vigor_col8_s100/brasil_pasture_vigor_col8_s100_year={ano}.csv'
    response = requests.get(url)
    if response.status_code == 200:
        if contador == 1:
            df_vpastagem = pd.read_csv(url)
            contador += 1
        else:
            df_vpastagem = pd.concat(
                [df_vpastagem,
                 (pd.read_csv(url))],
                 ignore_index=True            
            )
            contador += 1
    else:
        pass

df_vpastagem.to_csv(f'{raiz}/bronze_{fonte}_{dataset_name}.csv')


In [ ]:
# Tabelas relativas as áreas de pastagem no Brasil

contador = 1

for ano in range(2000, ano_anterior):
    url = f'https://s3.lapig.iesa.ufg.br/ows/country/BRASIL/csv/pasture_col8_s100/brasil_pasture_col8_s100_year={ano}.csv'
    response = requests.get(url)
    if response.status_code == 200:
        if contador == 1:
            df_pastagem = pd.read_csv(url)
            contador += 1
        else:
            df_pastagem = pd.concat(
                [df_pastagem,
                 (pd.read_csv(url))],
                 ignore_index=True            
            )
            contador += 1
    else:
        pass

df_pastagem.to_csv(f'{raiz}/bronze_{fonte}_pastagens.csv')

### Prata

In [19]:
# Área de pastagens

df_past_lapig = (
    pd.read_csv(
        f'{raiz}/bronze_{fonte}_pastagens.csv',
        usecols=[2, 8, 12],
        dtype={'ano': int, 'geocod_mun':int, 'classe': str, 'area_past_ha': float}
    )
    .rename(columns={
                    'ano': 'data',
                    'geocod_mun': 'geocodigo',
                    'area_past_ha': 'area'
                    })
    .query('area != 0.0') # Exclusão de possíveis registros que possuam valor zerado
    .assign(
            data = lambda df: pd.to_datetime(df.data, format='%Y'), # Conversão de número inteiro para formato de data seguindo a ISO 8601
            area = lambda x: x.area_ha/1000000 # Normalização da área
    ) 
    .sort_values(by=['geocodigo', 'data']) # Organização dos dados
    .set_index(['geocodigo', 'data'])
)
df_past_lapig.to_parquet(f'{raiz}/prata_{fonte}_pastagens_municipio.parquet')

df_past_lapig = (
    df_past_lapig
    .reset_index()
    .assign(geocodigo_uf = lambda x: x.geocodigo.astype(str).str[:2].astype(int))
    .groupby(['geocodigo_uf', 'data'])
    .agg({'area':sum})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)
df_past_lapig.to_parquet(f'{raiz}/prata_{fonte}_pastagens_uf.parquet')

In [20]:
df_past_mapbiomas = (
    pd.read_parquet(f'{raiz}/prata_mapbiomas_cobertura_solo_uf.parquet')
    .reset_index()
    .query('classe_solo in (15, 18, 21) & data >= 2000')
    .groupby(['data'])
    .agg({'area': sum})
)

In [21]:
df_past_mapbiomas

,area
data,
2000-01-01,186.738581
2001-01-01,188.118752
2002-01-01,190.316732
2003-01-01,193.954943
2004-01-01,195.786152
2005-01-01,196.811348
2006-01-01,196.978095
2007-01-01,196.888249
2008-01-01,196.985544


In [43]:
df_uf

area
geocodigo data       classe                 
11        2000-01-01 Ausente        2.673517
                     Intermediário  1.790422
                     Severa         0.280312
          2001-01-01 Ausente         2.83441
                     Intermediário  1.875347
...                                      ...
53        2021-01-01 Intermediário  0.035911
                     Severa         0.059063
          2022-01-01 Ausente        0.012446
                     Intermediário  0.035458
                     Severa         0.063618

[1863 rows x 1 columns]

In [40]:
# Área de pastagens a partir do nível de degradação

df_munic = (pd.read_csv(f'{raiz}/bronze_{fonte}_{dataset_name}.csv',
                        usecols=[2, 7, 11, 13],
                        dtype={'ano': int, 'geocod_mun':int, 'classe': str, 'area_past_ha': float})
                        .rename(columns={
                            'ano': 'data',
                            'geocod_mun': 'geocodigo',
                            'area_past_ha': 'area'
                            })
                        .query('area != 0.0') # Exclusão de possíveis registros que possuam valor zerado
                        .assign(data = lambda df: pd.to_datetime(df.data, format='%Y'), # Conversão de número inteiro para formato de data seguindo a ISO 8601
                                area = lambda x: x.area/1000000) 
                        .sort_values(by=['geocodigo', 'data']) # Organização dos dados
                        .set_index(['geocodigo', 'data', 'classe'])
            )
df_munic.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_municipio.parquet')

df_uf = (df_munic
         .reset_index()
         .assign(geocodigo_uf = lambda x: x.geocodigo.astype(str).str[:2].astype(int))
         .groupby(['geocodigo_uf', 'data', 'classe'])
         .agg({'area':sum})
         .rename_axis(index={'geocodigo_uf': 'geocodigo'})
         .convert_dtypes()
        )
df_uf.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_uf.parquet')